# Lab / HomeWork
## [Artificial Neural Network (ANN)]
## Feedfroward Neural Network (FFNN)

In [40]:
import numpy as np
import pandas as pd

### Data extraction and choice amount of neurons


In [41]:
data = pd.read_csv("data_ffnn_3classes.txt",header = None,sep=" ")

V_neuron= 10
W_neuron = 3

### Generation matrix Y


In [42]:
data_y = data[2]
length_y = len(data_y)
matrix_y = np.zeros((length_y,3))

def matrixY(y, length):
    for i in range(length):
        if y[i] == 0.0:
            matrix_y[i][0] = 1
        elif y[i] == 1.0:
            matrix_y[i][1] = 1
        elif y[i] == 2.0:
            matrix_y[i][2] = 1
            
matrixY(data_y, length_y)
print("dimension matrix y: " + str(matrix_y.shape))

dimension matrix y: (71, 3)


## Forward Propagation

### Input data matrix []<sub> I x N
I = Number of elements

N = Number of features

In [43]:
def input_matrix_x():
    return np.array([data[0], data[1]])
    
matrix_x = input_matrix_x().transpose()
print("dimension matrix_x: " + str(matrix_x.shape))

dimension matrix_x: (71, 2)


### Extended input data matrix []<sub> I x (N+1)

In [44]:
def x_1_matrix(x_mat):
    ones = np.ones(((np.size(x_mat, 0)),1))
    return np.append(ones, x_mat, axis=1)

matrix_x_extended = x_1_matrix(matrix_x)
print("dimension matrix_x_extended: " + str(matrix_x_extended.shape))

dimension matrix_x_extended: (71, 3)


### Extended input data matrix 2 []<sub> I x K

K = number of hidden neurons

v_matrix = []<sub> (N+1) x K

In [45]:
def x_2_matrix(x_mat_ext, v_neuron):
    return x_mat_ext.dot(v_neuron)

v_neutron_matrix = np.random.rand(np.size(matrix_x_extended, 1), V_neuron)
matrix_x_extended_2 = x_2_matrix(matrix_x_extended, v_neutron_matrix)
print("dimension matrix_x_extended_2: " + str(matrix_x_extended_2.shape))

dimension matrix_x_extended_2: (71, 10)


### Hidden F matrix []<sub> I x K


In [46]:
def f_matrix(x_mat_ext_2):
    return 1/(1 + np.exp(-x_mat_ext_2))
    
matrix_f = f_matrix(matrix_x_extended_2)
print("dimension matrix_f: " + str(matrix_f.shape))

dimension matrix_f: (71, 10)


### Extended hidden f matrix []<sub> I x (K+1)

In [47]:
def f_1_matrix(f_mat):
    ones = np.ones((np.size(f_mat,0),1))
    return np.append(ones, f_mat,axis=1)

matrix_f_extended = f_1_matrix(matrix_f)
print("dimension matrix_f_extended: " + str(matrix_f_extended.shape))

dimension matrix_f_extended: (71, 11)


### Extended hidden f matrix 2 []<sub> I x J

J = output variable dimension

w_matrix = []<sub> (K+1) x J

In [48]:
def f_2_matrix(f_mat_1, w_neuron):
    return f_mat_1.dot(w_neuron)

w_neutron_matrix = np.random.rand(V_neuron+1, W_neuron)
matrix_f_extended_2 = f_2_matrix(matrix_f_extended, w_neutron_matrix)
print("dimension matrix_f_extended_2: " + str(matrix_f_extended_2.shape))

dimension matrix_f_extended_2: (71, 3)


### G matrix []<sub> I x J


In [49]:
def g_matrix(f_mat_ext_2):
    return 1/(1 + np.exp(-f_mat_ext_2))

matrix_g = g_matrix(matrix_f_extended_2)
print("dimension matrix_g: " + str(matrix_g.shape))

dimension matrix_g: (71, 3)


### Sum of squared errors (SSE)

In [50]:
def sum_squared_errors_calculation(g_mat, y_mat):
    squares = (g_mat - y_mat ) ** 2
    return np.sum(squares)/2

sum_squared_errors = sum_squared_errors_calculation(matrix_g, matrix_y)
print("sum_squared_errors: " + str(sum_squared_errors))

sum_squared_errors: 70.20098043333145


## Backwards propagation

### Correction of w matrix

In [51]:
def w_correction(G,Y,F,W,alpha):
    w_matrix_new = np.zeros((np.size(W,0),np.size(W,1)))
    # w of size (k+1)xJ
    for k in range(np.size(W,0) - 1):
        for j in range(np.size(W,1)):
            w = W[k][j]
            err = 0
            for i in range(np.size(G,1)):
                g = G[i][j]
                y = Y[i][j]
                f = F[i][k]
                
                err += (g-y) * g * (1-g) * f
            w_matrix_new[k][j] = w - (alpha*err)
    return w_matrix_new

new_w_mat = w_correction(matrix_g, matrix_y, matrix_f_extended, w_neutron_matrix, 0.5)
print(new_w_mat)

[[0.32901861 0.60830043 0.10200596]
 [0.40114545 0.8878664  0.47741999]
 [0.31398826 0.51802625 0.66812421]
 [0.37502873 0.30963634 0.97267913]
 [0.27299515 0.9308722  0.26894092]
 [0.72518086 0.51530811 0.93922216]
 [0.55855741 0.81890513 0.36509121]
 [0.82370815 0.18382282 0.01176581]
 [0.10144549 0.04586955 0.54190455]
 [0.26881015 0.74885849 0.55412551]
 [0.         0.         0.        ]]


### Correction of v matrix 

In [52]:
def v_correction(g_mat, y_mat, f_mat, w_mat, x_mat_ext, v_mat, alpha):
    v_matrix_new = np.zeros((np.size(v_mat, 0),np.size(v_mat, 1)))
    for n in range(np.size(x_mat_ext, 1)):
        for k in range(np.size(v_mat, 1)):
            v = v_mat[n][k]
            err = 0
            for i in range(np.size(g_mat, 0)):
                f = f_mat[i][k]
                x_ext = x_mat_ext[i][n]
                
                for j in range(np.size(g_mat, 1)):
                    g = g_mat[i][j]
                    y = y_mat[i][j]
                    w = w_mat[k][j]
                    err += (g-y) * g * (-g) * w * f * (1-f) * x_ext
            v_matrix_new[n][k] = v - (alpha*err)
    return v_matrix_new

new_v_mat = v_correction(matrix_g, matrix_y, matrix_f, w_neutron_matrix, matrix_x_extended, v_neutron_matrix, 0.5)
print(new_v_mat)
        

[[ 1.04382064  7.02716209  1.32512208  0.74984572  0.82711475  4.22222422
   0.84640014  0.80180081  0.59809597  0.95051231]
 [ 2.19474305 19.6683886   2.70776444  1.39939878  2.17455699  7.89216366
   1.88791685  1.79240947  1.55248243  1.80974262]
 [ 2.13471011 16.23425577  3.23914836  1.65509433  2.37198923  9.28701777
   2.00010726  1.5748997   1.80649572  2.13189465]]
